DATA: https://www.kaggle.com/datasets/thomasnibb/amsterdam-house-price-prediction

**IMPORT LIBRARIES**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Read dataset
data_path = '/kaggle/input/amsterdam-house-price-prediction/HousingPrices-Amsterdam-August-2021.csv'
df = pd.read_csv(data_path)

# We drop redundant columns
df = df.drop(columns = ['Unnamed: 0', 'Address', 'Zip'], axis = 1)
df.head()

,Price,Area,Room,Lon,Lat
0,685000.0,64,3,4.907736,52.356157
1,475000.0,60,3,4.850476,52.348586
2,850000.0,109,4,4.944774,52.343782
3,580000.0,128,6,4.789928,52.343712
4,720000.0,138,5,4.902503,52.410538


**PREPROCESS THE DATA**

In [3]:
# Check NaN values
print(df.isnull().sum())
df = df.dropna()

Price    4
Area     0
Room     0
Lon      0
Lat      0
dtype: int64


In [4]:
# Standardize the data
scaler = MinMaxScaler()
cols_to_normalize = ['Price', 'Area', 'Room', 'Lon', 'Lat']

for col in cols_to_normalize:
    df[col] = scaler.fit_transform(df[[col]])

**TRAIN/TEST DATA SPLIT**

In [5]:
# Input/Target split
X = df.drop(columns = ['Price'])
y = df['Price']

# Train/Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 42
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((644, 4), (276, 4), (644,), (276,))

**XGBOOST MODEL**

In [6]:
# We implement XGBoost model
xg_reg = xgb.XGBRegressor(
    seed = 42,
    learning_rate = 0.01,
    n_estimators = 102,
    max_depth = 3
)

xg_reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=102, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [7]:
# Predictions based on the model
predictions = xg_reg.predict(X_test)

In [8]:
# Evaluate the model
def evaluation(data):
    predictions = xg_reg.predict(data)
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    print('Evaluation results on test set:')
    print(f'Mean Absolute Error: {mae}')
    print(f'Mean Squared Error: {mse}')

evaluation(X_test)

Evaluation results on test set:
Mean Absolute Error: 0.03531575046846068
Mean Squared Error: 0.003686753763325665
